In [ ]:
import JSON
using Plots
using HDF5
include("tools.jl")
include("get_grid.jl")
include("get_params.jl")
include("disturbances.jl")

In [ ]:
border = import_border("data/border.json")
dx = 20
Nx, Ny, xrange, yrange, isgrid, isborder, n = get_grid(border, Float64(dx))
x = repeat(reshape(xrange,1,Nx),Ny,1)
y = repeat(reshape(yrange,Ny,1),1,Nx)
x_for_bx = repeat(reshape(xrange,1,Nx),Ny,1) .- dx/2
y_for_bx = repeat(reshape(yrange,Ny,1),1,Nx)
x_for_by = repeat(reshape(xrange,1,Nx),Ny,1)
y_for_by = repeat(reshape(yrange,Ny,1),1,Nx) .+ dx/2

In [ ]:
sigma_l = 1E2
#sigma_l = 40.
recompute_param = true

if(recompute_param)
    bx, by, p, m, d = get_params(isborder, sigma_l, "data/pantagruel.h5", "grid_params_20.h5")
else
    bx, by, p, m, d = get_params("grid_params_20.h5")
end


dP = -9.0
sigma = 30.0
location = [-1500., -900.]
dp = local_disturbance(isgrid, xrange, yrange, location, dP, sigma)
do_plot(isgrid, dp)

# u*dt^2/dx^2 should be << 1
u = zeros(Ny,Nx)
for j=2:Nx-1
    for i=2:Ny-1
        u[i,j] = (by[i-1,j] + by[i,j] + bx[i,j] + bx[i,j+1]) / m[i,j]
    end
end
u[.!isgrid] .= 0;
maximum(u)

In [ ]:
interval = 50
dt = 0.0001
Ndt = 5000

gamma = d ./ m

omegas = zeros(Ny,Nx,1 + Int64(ceil(Ndt/interval)))
thetas = zeros(Ny,Nx,1 + Int64(ceil(Ndt/interval)))
th_new = zeros(Ny,Nx)
reset = true 
if(reset)
    th_old = zeros(Ny,Nx)
    th = zeros(Ny,Nx)
    omegas[:,:,1] = th_old
else
    #data = h5read("sim.h5","/")
    #th_old = data["thetas"][:,:,end]
    #th_old[.!isgrid] .= NaN
    #th = data["thetas"][:,:,end]
    #omegas[:,:,1] = data["omegas"][:,:,end]
end
println(maximum(abs.(th_old[isgrid]))) 
ts = zeros(1 + Int64(ceil(Ndt/interval)))
chi = 1 .+ gamma*dt/2

@time begin
    for k in 1:Ndt
        for i in 2:Ny-1
            for j in 2:Nx-1
                if(isborder[i,j])
                    id = (n[:,1] .== i) .& (n[:,2] .== j)
                    nx = n[id,3][1]
                    ny = n[id,4][1]
                    th_new[i,j] = (nx * ( bx[i,j] * th[i,j-1] - bx[i,j+1] * th[i,j+1]) +
                        ny * (by[i-1,j] * th[i-1,j] - by[i,j] * th[i+1,j])) /
                        (nx * (bx[i,j] - bx[i,j+1]) + ny * (by[i-1,j] - by[i,j]))
                elseif(isgrid[i,j])
                    bij = (by[i-1,j] + by[i,j] + bx[i,j] + bx[i,j+1])
                    th_new[i,j] = (2 - bij / m[i,j] * dt^2 / dx^2) / chi[i,j] * th[i,j] + 
                        (gamma[i,j] * dt/2 - 1) /chi[i,j] * th_old[i,j] + 
                        dt^2 / dx^2 / chi[i,j] / m[i,j] * 
                        (by[i,j] * th[i+1,j] + by[i-1,j] * th[i-1,j] +
                        bx[i,j+1] * th[i,j+1] + bx[i,j] * th[i,j-1]) +
                        dt^2 / chi[i,j] / m[i,j] * (p[i,j] + dp[i,j])
                end
            end
        end   
        if(mod(k,interval) == 0)
            println(k)
            temp = (th_new-th) / dt
            omegas[:,:,Int64(k/interval) + 1] = temp
            thetas[:,:,Int64(k/interval) + 1] = th_ne
            ts[Int64(k/interval) + 1] = k*dt
        end
        th_old = copy(th)
        th = copy(th_new)
    end
end

In [ ]:
@gif for i=1:size(omegas,3)
    do_plot(isgrid, omegas[:,:,i])
end